In [1]:
import pandas as pd
import tqdm
import numpy as np

In [2]:
genomes = pd.read_csv("metadata.tsv",sep="\t")

/home/theo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
genomes = genomes[['Collection date','Location','Pango lineage']]
parts = genomes.Location.str.split("/",expand=True)
genomes['country']=parts[1].str.strip()
genomes=genomes.drop(columns="Location").rename(columns={'Pango lineage':'Lineage','Collection date':"date"})
lineages = genomes[genomes['date'].str.len()>7].copy()
lineages.loc[lineages['country']=="USA","country"]="United States"
selected_lineages = ["A","B","B.1.177","B.1.1.7","B.1.351","P.1","B.1.617","B.1.525"]
lineages=lineages[lineages.Lineage.notna()].copy()

In [4]:
selected_lineages = ["A","B","B.1.177","B.1.1.7","B.1.351","P.1","B.1.617","B.1.525"]

In [5]:
lineages['new_lineage'] = 'other'
for lin in selected_lineages:
    lineages.loc[lineages['Lineage'].str.startswith(f"{lin}."),'new_lineage']=lin
    lineages.loc[lineages['Lineage'] == lin,'new_lineage']=lin
lineages = lineages.drop(columns="Lineage").rename(columns={'new_lineage':'Lineage'})

In [6]:
#countries = sorted(set(lineages.country.tolist()).intersection(cases['location'].tolist()))
countries = sorted([x for x in set(lineages.country.tolist()) if x is not None])
lineages = lineages[lineages.date.str.match(r'202.-[0-9][0-9]-[0-9][0-9]')==True]
lineages['date'] = pd.to_datetime(lineages['date'])
lineages.Lineage=  lineages.Lineage.astype("category")

In [7]:
lineages

date    country Lineage
5       2020-03-23  Australia       B
6       2020-03-28  Australia       B
7       2020-03-28  Australia       B
8       2020-03-28  Australia       B
9       2020-03-28  Australia       B
...            ...        ...     ...
1194347 2020-12-18  Venezuela   other
1194348 2020-12-18  Venezuela   other
1194349 2020-12-18  Venezuela   other
1194350 2020-12-18  Venezuela   other
1194351 2020-12-18  Venezuela   other

[1166480 rows x 3 columns]

In [9]:
import tqdm
import numpy as np

min_date="2020-03-01"
max_date="2021-04-5"

dfs = []

for country in tqdm.tqdm(countries+['overview']):
    if country=='overview':
        country_set= lineages
    else:
        country_set = lineages[lineages['country']==country]
    for date in  pd.date_range(min_date,max_date):
        restr = country_set[
np.logical_and( country_set["date"] > date- pd.Timedelta(1,unit="W"), country_set["date"] <= date + pd.Timedelta(1,unit="W"))]
        counts = restr.Lineage.value_counts()
        props = counts/counts.sum()
        if counts.sum()<3:
            props[props>-5]=pd.NA
        
        new_df = pd.DataFrame({'lambda':counts/2,'p':props,'ltla':country,'date':date})
        dfs.append(new_df)




100%|██████████| 176/176 [02:37<00:00,  1.11it/s]


In [10]:
big_df = pd.concat(dfs)
big_df = big_df.reset_index().rename(columns={'index':'lineage'})
big_df = big_df.melt(value_vars=['lambda','p'],id_vars=['date','ltla','lineage'], value_name = "mean",var_name="parameter")
big_df['lower'] = big_df['mean']
big_df['upper'] = big_df['mean']
big_df=big_df.rename(columns = {'ltla':'lad19cd'})
big_df.to_csv("full_data_table.csv")